In [7]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from tqdm import tqdm
#from NaiveStemmer.java import NaiveStemmer

ensure_pyterrier_is_loaded()
tira = Client()

In [8]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [12]:
class NaiveStemmer(pt.TerrierStemmer):
    def stem(self, token:str):
        cutLength = len(token) // 3
        cutLength = min(cutLength, 5)
        return token[0:len(token) - cutLength]

class CombinedStemmer(pt.TerrierStemmer):
    def stem(self, token:str):
        token = pt.index.TerrierStemmer.porter.stem(token)
        nSt = NaiveStemmer()
        return nSt.stem(token)

TypeError: NaiveStemmer: cannot extend enumeration 'TerrierStemmer'

In [ ]:
naiveStemmer = NaiveStemmer()
combinedStemmer = CombinedStemmer()


iter_indexer0 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = None , meta={'docno': 100, 'text': 4096})
indexref0 = iter_indexer0.index(tqdm(pt_dataset.get_corpus_iter()))
iter_indexer1 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = naiveStemmer , meta={'docno': 100, 'text': 4096})
indexref1 = iter_indexer1.index(tqdm(pt_dataset.get_corpus_iter()))
iter_indexer2 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = combinedStemmer , meta={'docno': 100, 'text': 4096})
indexref2 = iter_indexer2.index(tqdm(pt_dataset.get_corpus_iter()))

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-inputs.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 39.4M/39.4M [00:00<00:00, 50.8MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90188/126958 [00:24<00:06, 5443.68it/s]

10:29:59.256 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


...

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:21<00:00, 5784.49it/s] 
100%|██████████| 126958/126958 [00:21<00:00, 5785.03it/s]


10:30:37.858 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 89940/126958 [00:15<00:05, 6267.36it/s]

10:30:57.075 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:21<00:00, 5950.52it/s]
100%|██████████| 126958/126958 [00:21<00:00, 5951.11it/s]


In [ ]:
bm25NoStemmer = pt.BatchRetrieve(indexref0, wmodel="BM25")
bm25NaiveStemmer = pt.BatchRetrieve(indexref1, wmodel="BM25")
bm25CombinedStemmer = pt.BatchRetrieve(indexref2, wmodel="BM25")

In [ ]:
#pt_dataset.get_topics('text').head(3)

#print('Now we do the retrieval...')
run0 = bm25NoStemmer(pt_dataset.get_topics('text'))
run1 = bm25NaiveStemmer(pt_dataset.get_topics('text'))
run2 = bm25CombinedStemmer(pt_dataset.get_topics('text'))

#print('Done. Here are the first 10 entries of the run')
#run.head(10)

persist_and_normalize_run(run1, system_name='bm25_no_stemmer', default_output='../runs/noStemmer')
persist_and_normalize_run(run1, system_name='bm25_naive_stemmer', default_output='../runs/naiveStemmer')
persist_and_normalize_run(run2, system_name='bm25_combined_stemmer', default_output='../runs/combinedStemmer')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs/noStemmer".
Done. run file is stored under "../runs/noStemmer/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs/naiveStemmer".
Done. run file is stored under "../runs/naiveStemmer/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs/combinedStemmer".
Done. run file is stored under "../runs/combinedStemmer/run.txt".
